### import and create client for management service for user management

In [1]:
from instill.configuration import global_config
# setup target instance
global_config.set_default(
    url="localhost:8080",
    token="", # put your own api_token here
    secure=False,
)

# get the client with all services
from instill.clients import get_client
client = get_client()

E1208 03:57:04.168224000 7969234624 hpack_parser.cc:999]               Error parsing 'content-type' metadata: invalid value
E1208 03:57:04.170813000 7969234624 hpack_parser.cc:999]               Error parsing 'grpc-status' metadata: not an integer
2023-12-08 03:57:04,171.171 WARNING  Instill VDP is not serving, VDP functionalities will not work


### make sure the mgmt service is serving

In [3]:
client.mgmt_service.is_serving()

True

### make sure the model service is serving

In [4]:
client.mgmt_service.is_serving()

True

### import model resource class for creating model instance

In [5]:
from instill.resources.model import GithubModel

### import protobuf definition files for easier type reference

In [6]:
import instill.protogen.model.model.v1alpha.model_pb2 as model_interface
import instill.protogen.model.model.v1alpha.task_classification_pb2 as classification

### define a sample model configuration

In [12]:
github_model = {
    "model_name": "mobilenetv2",
    "model_repo": "instill-ai/model-mobilenetv2-dvc",
    "model_tag": "v1.0-cpu",
}

### create model

In [10]:
model = GithubModel(
            client=model_client,
            name=github_model["model_name"],
            model_repo=github_model["model_repo"],
            model_tag=github_model["model_tag"],
        )

2023-09-02 03:33:45,892.892 INFO     mobilenetv2 creating...
2023-09-02 03:33:46,927.927 INFO     mobilenetv2 creating...
2023-09-02 03:33:47,958.958 INFO     mobilenetv2 creating...


### check the state of the newly created model, state should be offline

In [11]:
model.get_state()

'STATE_OFFLINE'

### now deploy the model

In [12]:
model.deploy()

2023-09-02 03:33:51,649.649 INFO     mobilenetv2 deploying...
2023-09-02 03:33:52,669.669 INFO     mobilenetv2 deploying...
2023-09-02 03:33:53,695.695 INFO     mobilenetv2 deploying...
2023-09-02 03:33:54,720.720 INFO     mobilenetv2 deploying...
2023-09-02 03:33:55,752.752 INFO     mobilenetv2 deploying...
2023-09-02 03:33:56,779.779 INFO     mobilenetv2 deploying...
2023-09-02 03:33:57,799.799 INFO     mobilenetv2 deploying...
2023-09-02 03:33:58,815.815 INFO     mobilenetv2 deploying...
2023-09-02 03:33:59,833.833 INFO     mobilenetv2 deploying...
2023-09-02 03:34:00,843.843 INFO     mobilenetv2 deploying...
2023-09-02 03:34:01,853.853 INFO     mobilenetv2 deploying...
2023-09-02 03:34:02,883.883 INFO     mobilenetv2 deploying...
2023-09-02 03:34:03,894.894 INFO     mobilenetv2 deploying...
2023-09-02 03:34:04,940.940 INFO     mobilenetv2 deploying...
2023-09-02 03:34:05,951.951 INFO     mobilenetv2 deploying...


True

### check the state again, should be online

In [13]:
model.get_state()

'STATE_ONLINE'

### define a couple of inputs for calssification task

In [14]:
task_inputs = [
    model_interface.TaskInput(
        classification=classification.ClassificationInput(
            image_url="https://artifacts.instill.tech/imgs/dog.jpg"
        )
    ),
    model_interface.TaskInput(
        classification=classification.ClassificationInput(
            image_url="https://artifacts.instill.tech/imgs/tiff-sample.tiff"
        )
    ),
]

### trigger inference with the inputs

In [15]:
model(task_inputs=task_inputs)

[classification {
  category: "golden retriever"
  score: 0.896806
}
, classification {
  category: "notebook"
  score: 0.511446
}
]

### after we are done with the model, simply delete it

In [16]:
print(model_client.get_model("mobilenetv2"))

name: "users/instill-ai/models/mobilenetv2"
uid: "0b9c26f2-9d51-43ed-ae0f-30b3f280e7be"
id: "mobilenetv2"
description: ""
model_definition: "model-definitions/github"
task: TASK_CLASSIFICATION
state: STATE_ONLINE
visibility: VISIBILITY_PUBLIC
user: "users/4767b74d-640a-4cdf-9c6d-7bb0e36098a0"
create_time {
  seconds: 1693596826
  nanos: 486756000
}
update_time {
  seconds: 1693596828
  nanos: 998432000
}



In [17]:
del model
model_client.get_model("mobilenetv2")

2023-09-02 03:34:09,550.550 ERROR    <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.NOT_FOUND
	details = "[db] getUserModel error: record not found"
	debug_error_string = "UNKNOWN:Error received from peer ipv6:%5B::1%5D:9080 {grpc_message:"[db] getUserModel error: record not found", grpc_status:5, created_time:"2023-09-02T03:34:09.548676+08:00"}"
>
Traceback (most recent call last):
  File "/Users/heiru/Projects/instill/python-sdk/instill_sdk/utils/error_handler.py", line 12, in func_wrapper
    return func(*args, **kwargs)
  File "/Users/heiru/Projects/instill/python-sdk/instill_sdk/clients/model.py", line 272, in get_model
    return self._stub.GetUserModel(
  File "/Users/heiru/miniconda3/envs/sdk/lib/python3.8/site-packages/grpc/_channel.py", line 946, in __call__
    return _end_unary_response_blocking(state, call, False, None)
  File "/Users/heiru/miniconda3/envs/sdk/lib/python3.8/site-packages/grpc/_channel.py", line 849, in _end_unary_response_blocking
   

model.delete()